Датасет по лучшим трекам года по версии Billboard

In [276]:
import requests
import json
from bs4 import BeautifulSoup
from IPython.display import display

import pandas as pd

In [142]:
def billboard_chart_page(year):
    url = 'https://www.billboard.com/charts/year-end/{year}/hot-100-songs'.format(year=year)
    response = requests.get(url)
    content = response.content.decode()
    return content

In [143]:
def parse_billboard(content):
    page = BeautifulSoup(content, 'lxml')
    artists = [tag.get_text().strip() for tag in page.body.find_all('div', class_='ye-chart-item__artist')]
    songs = [tag.get_text().strip() for tag in page.body.find_all('div', class_='ye-chart-item__title')]
    return artists, songs

In [144]:
def track_list(artists, songs):
    return ['{} - {}'.format(artist, song) for artist, song in zip(artists, songs)]

Извлекаем названия всех треков

In [145]:
available_years = range(2007, 2018)
tracks = dict().fromkeys(available_years)
for year in available_years:
    tracks[year] = track_list(*parse_billboard(billboard_chart_page(year)))

В 2011 году почему-то выкачиваются 99 треков, добавим пустой трек

In [146]:
display([(k, len(v)) for k, v in tracks.items()])
if len(tracks[2011]) < 100:
    tracks[2011].append('')

[(2007, 100),
 (2008, 100),
 (2009, 100),
 (2010, 100),
 (2011, 99),
 (2012, 100),
 (2013, 100),
 (2014, 100),
 (2015, 100),
 (2016, 100),
 (2017, 100)]

In [147]:
df_tracks = pd.DataFrame(tracks)
df_tracks.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Beyonce - Irreplaceable,Flo Rida Featuring T-Pain - Low,The Black Eyed Peas - Boom Boom Pow,Ke$ha - TiK ToK,Adele - Rolling In The Deep,Gotye Featuring Kimbra - Somebody That I Used ...,Macklemore & Ryan Lewis Featuring Wanz - Thrif...,Pharrell Williams - Happy,Mark Ronson Featuring Bruno Mars - Uptown Funk!,Justin Bieber - Love Yourself,Ed Sheeran - Shape Of You
1,Rihanna Featuring Jay-Z - Umbrella,Leona Lewis - Bleeding Love,Lady Gaga - Poker Face,Lady Antebellum - Need You Now,LMFAO Featuring Lauren Bennett & GoonRock - Pa...,Carly Rae Jepsen - Call Me Maybe,Robin Thicke Featuring T.I. + Pharrell - Blurr...,Katy Perry Featuring Juicy J - Dark Horse,Ed Sheeran - Thinking Out Loud,Justin Bieber - Sorry,Luis Fonsi & Daddy Yankee Featuring Justin Bie...
2,Gwen Stefani Featuring Akon - The Sweet Escape,Alicia Keys - No One,Lady Gaga Featuring Colby O'Donis - Just Dance,"Train - Hey, Soul Sister",Katy Perry - Firework,fun. Featuring Janelle Monae - We Are Young,Imagine Dragons - Radioactive,John Legend - All Of Me,Wiz Khalifa Featuring Charlie Puth - See You A...,Drake Featuring WizKid & Kyla - One Dance,Bruno Mars - That's What I Like
3,Fergie - Big Girls Don't Cry,Lil Wayne Featuring Static Major - Lollipop,The Black Eyed Peas - I Gotta Feeling,Katy Perry Featuring Snoop Dogg - California G...,Katy Perry Featuring Kanye West - E.T.,Maroon 5 Featuring Wiz Khalifa - Payphone,Baauer - Harlem Shake,Iggy Azalea Featuring Charli XCX - Fancy,Fetty Wap - Trap Queen,Rihanna Featuring Drake - Work,Kendrick Lamar - Humble.
4,T-Pain Featuring Yung Joc - Buy U A Drank (Sha...,Timbaland Featuring OneRepublic - Apologize,Taylor Swift - Love Story,Usher Featuring will.i.am - OMG,"Pitbull Featuring Ne-Yo, Afrojack & Nayer - Gi...",Ellie Goulding - Lights,Macklemore & Ryan Lewis Featuring Ray Dalton -...,OneRepublic - Counting Stars,Maroon 5 - Sugar,twenty one pilots - Stressed Out,The Chainsmokers & Coldplay - Something Just L...


Сохраняем промежуточный результат

In [277]:
with open('tracks.json', 'w') as json_file:
    json.dump(tracks, json_file, indent=4)

Получаем URLы видео по запросу. Фильтруем дубликаты, сохраняя порядок

In [236]:
def youtube_search(query):
    response = requests.get(url='https://www.youtube.com/results', params=dict(search_query=query))
    content = response.content.decode()
    page = BeautifulSoup(content, 'lxml')
    endpoints = list(
        link['href'].replace('https://www.youtube.com', '').split('&')[0]
        for link in page.body.find_all('a') 
        if '/watch?v=' in link['href']
    )
    unique_endpoints = OrderedDict.fromkeys(endpoints).keys()
    video_urls = ['https://www.youtube.com{}'.format(endpoint) for endpoint in unique_endpoints]
    return video_urls

In [266]:
def find_video_clip(track):
    video_urls = youtube_search(track)
    if video_urls:
        return video_urls[0]
    else:
        return None

Youtube не отдает URLы, потому что забанил мою домашнюю сеть. Тут нужно прикрутить прокси, либо использовать иной ресурс для скачивания треков.

In [267]:
track_details = {}
for year, track_list in tracks.items():
    print('Year:', year)
    for i, track in enumerate(track_list):
        if i % 10 == 0:
            print('The number of processed videos:', i)
        
        youtube=find_video_clip(track)
        if youtube is None:
            youtube_id=None
        else:
            youtube_id=youtube.split('/watch?v=')[-1]
            
        track_details[track] = dict(
            year=year,
            name=track,
            youtube=youtube,
            youtube_id=youtube_id
        )

Year: 2007
The number of processed videos: 0
The number of processed videos: 10
The number of processed videos: 20
The number of processed videos: 30
The number of processed videos: 40
The number of processed videos: 50
The number of processed videos: 60
The number of processed videos: 70
The number of processed videos: 80
The number of processed videos: 90
Year: 2008
The number of processed videos: 0
The number of processed videos: 10
The number of processed videos: 20
The number of processed videos: 30
The number of processed videos: 40
The number of processed videos: 50
The number of processed videos: 60
The number of processed videos: 70
The number of processed videos: 80
The number of processed videos: 90
Year: 2009
The number of processed videos: 0
The number of processed videos: 10
The number of processed videos: 20
The number of processed videos: 30
The number of processed videos: 40
The number of processed videos: 50
The number of processed videos: 60
The number of processed v

Сохраняем промежуточный результат

In [280]:
with open('track_details.json', 'w') as json_file:
    json.dump(track_details, json_file, indent=4)

In [271]:
def download_audio_from_youtube(youtube_urls):
    if not youtube_urls:
        return
    
    ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download(youtube_urls)

In [281]:
youtube_urls = [track['youtube'] for track in track_details.values() if track['youtube']]
download_audio_from_youtube(youtube_urls)